In [4]:
!cp -r django_template todo_list

In [8]:
!rm -rf todo_list/

In [7]:
import os
import shutil

def start(project_name):
    if not project_name:
        print("Project name cannot be empty.")
        return False

    if os.path.exists(project_name):
        print(f"Directory '{project_name}' already exists. Please choose a different name or remove the existing directory.")
        return False

    shutil.copytree('django_template', project_name)
    print(f"Django template copied to '{project_name}' directory.")

    return True

In [9]:
name = 'todo_list'

In [10]:
start(name)

Django template copied to 'todo_list' directory.


True

In [15]:
!git grep django

In [16]:
!ls 

coding-agent.ipynb  django_template  todo_list


- tree
- grep
- read file
- write file
- bash

In [38]:
import os
import subprocess
from pathlib import Path


class AgentTools:
    def __init__(self, root_dir):
        self.root = Path(root_dir).resolve()
        self.skip_dirs = {
            ".git",
            ".venv",
            "venv",
            "__pycache__",
            ".mypy_cache",
            ".pytest_cache",
            ".idea",
            ".vscode"
        }

    def _safe(self, path):
        p = (self.root / path).resolve()
        if not str(p).startswith(str(self.root)):
            raise ValueError(f"Path escapes root: {p}")
        return p

    def tree(self, path=".", max_depth=None):
        """
        Return only files under `path`, relative to the repo root.
        Skips unwanted directories.
        """
        start = self._safe(path)
        results = []

        def walk(p, depth):
            if max_depth is not None and depth > max_depth:
                return

            for entry in p.iterdir():
                # Skip junk dirs entirely
                if entry.is_dir() and entry.name in self.skip_dirs:
                    continue

                # File? Add it (relative)
                if entry.is_file():
                    rel = entry.relative_to(self.root)
                    results.append(str(rel))

                # Folder? Recurse
                if entry.is_dir():
                    walk(entry, depth + 1)

        walk(start, 0)
        return results

    def grep(self, pattern, path=".", ignore_case=False):
        """Search for files containing pattern. Returns relative paths."""
        search_root = self._safe(path)
        matches = []

        if ignore_case:
            pattern = pattern.lower()

        for root, dirs, files in os.walk(search_root):
            # Remove skipped dirs from traversal
            dirs[:] = [d for d in dirs if d not in self.skip_dirs]

            for file in files:
                file_path = Path(root) / file
                rel = file_path.relative_to(self.root)

                try:
                    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                        for i, line in enumerate(f, 1):
                            hay = line.lower() if ignore_case else line
                            if pattern in hay:
                                matches.append((str(rel), i, line.rstrip()))
                except (OSError, UnicodeDecodeError):
                    continue

        return matches

    def read_file(self, path):
        p = self._safe(path)
        return p.read_text(encoding="utf-8", errors="replace")

    def write_file(self, path, text):
        p = self._safe(path)
        p.parent.mkdir(parents=True, exist_ok=True)
        p.write_text(text, encoding="utf-8")
        return True

    def execute_bash(self, command, timeout=30):
        """
        Run a bash command inside the repository root, automatically prefixing
        *each* subcommand with 'uv run' unless already present.

        Supports compounds like:
            a && b
            a || b
            a ; b ; c

        Args:
            command (str): Shell command to execute.
            timeout (int): Maximum time before aborting.

        Returns:
            tuple[int, str, str]: (exit_code, stdout, stderr)
        """

        # Operators that separate multiple commands
        separators = ["&&", "||", ";"]

        # Detect which separator is used (simple, but enough for practical use)
        sep_used = None
        for sep in separators:
            if sep in command:
                sep_used = sep
                break

        if sep_used:
            # Split into individual commands
            parts = [c.strip() for c in command.split(sep_used)]

            # Prefix each command with uv run if needed
            processed = []
            for part in parts:
                if part.startswith("uv run "):
                    processed.append(part)
                else:
                    processed.append(f"uv run {part}")

            final_cmd = f" {sep_used} ".join(processed)

        else:
            # Single command
            cmd = command.strip()
            if not cmd.startswith("uv run "):
                cmd = f"uv run {cmd}"
            final_cmd = cmd

        # Execute
        try:
            proc = subprocess.run(
                final_cmd,
                shell=True,
                capture_output=True,
                text=True,
                cwd=str(self.root),
                timeout=timeout
            )
            return proc.returncode, proc.stdout, proc.stderr

        except subprocess.TimeoutExpired as e:
            return -1, e.stdout or "", f"Timeout after {timeout}s"



In [39]:
agent_tools = AgentTools('todo_list')

In [ ]:
import os
del os.environ['VIRTUAL_ENV']

In [40]:
agent_tools.tree()

['static/css/styles.css',
 'Makefile',
 'README.md',
 '.gitignore',
 'db.sqlite3',
 'templates/base.html',
 'pyproject.toml',
 'myproject/asgi.py',
 'myproject/__init__.py',
 'myproject/settings.py',
 'myproject/wsgi.py',
 'myproject/urls.py',
 'uv.lock',
 'myapp/__init__.py',
 'myapp/apps.py',
 'myapp/templates/home.html',
 'myapp/tests.py',
 'myapp/views.py',
 'myapp/migrations/__init__.py',
 'myapp/models.py',
 'myapp/admin.py',
 'manage.py']

In [46]:
agent_tools.execute_bash('python manage.py makemigrations && python manage.py migrate') 

(0,
 'No changes detected\nOperations to perform:\n  Apply all migrations: admin, auth, contenttypes, sessions\nRunning migrations:\n  No migrations to apply.\n',
 '')